# Setting up a venv-sandboxed Python kernel

This notebooks explains how to set up a new venv-sandboxed Python kernel and make it available in your JupyterLab environment, assuming that you are running this notebook in a standard linux-based environment and a regular (non-root) using running commands in a bash terminal. 

> This is meant to provide helpful hints to get you over the hump, not as a comprehensive tutorial. This may not work for your environment!

> You might also want to consult the [official IPython kernel installation documentation](https://ipython.readthedocs.io/en/stable/install/kernel_install.html) for a more general explanation. 

## Creating and installing a new venv-sandboxed Python kernel

From a bash terminal, `cd` to your home directory, and create a new hidden directory in which you will put all your virtual environments.

```
user@host:~$ mkdir .virtualenvs
```

Then, create a new venv (the venv in this example is named "foo", because of course it is).

```
user@host:~$ python3 -m venv .virtualenvs/foo
```

Activate the new venv. Once activated, the venv name is displayed in parentheses at the front of your command prompt, as shown below.

```
user@host:~$ source .virtualenvs/foo/bin/activate
(foo) user:host:~$
```

Use `pip` to install ipykernel inside your new venv.

```
(foo) user@host:~$ pip install ipykernel
```

Install your new ipykernel into your user kernel library, and give it a nice name that will make sense when you see the new kernel listed in the JupyterLab interface.

```
(foo) user@host:~$ python -m ipykernel install --user --name foo --display-name "Python 3 (venv:foo)"
```

Deactivate the venv in your terminal when you are done.

```
(foo) user@host:~$ deactivate
```

When you open a new Launcher tab in JupyterLab, you should see the new venv-sandboxed Python kernel listed next to the default system Python kernel.

To display a list to confirm which jupyter kernels are currently installed in your environment, you can run the following command from a terminal.

```
user@host:~$ jupyter kernelspec list
```

To uninstall the "foo" kernel, run the following command.

```
user@host:~$ jupyter kernelspec uninstall foo
```

This only uninstalls the jupyter kernel. It does not remove the venv containing the kernelspec from your system. To delete a venv, just delete the directory it is contained in.

```
user@host:~$ rm -rf .virtualenvs/foo
```

## Installing Python packages

There are multiple ways to install Python packages into your new venv, such that your venv-sandboxed Python kernel will pick these up when you run import commands in your code.

The easiest way is to use built-in `%pip` IPython magic commands from a JupyterLab notebook (or console) that is running your venv-sandboxed kernelspec. 

For example, to install the `dev` branch of the `ws3` package in the new "foo" venv you could run this line of code from inside the kernel.

In [1]:
%pip install -U git+https://github.com/gparadis/ws3@dev

  Cloning https://github.com/gparadis/ws3 (to revision dev) to /tmp/pip-req-build-n84lx90_
  Running command git clone --filter=blob:none --quiet https://github.com/gparadis/ws3 /tmp/pip-req-build-n84lx90_
  Resolved https://github.com/gparadis/ws3 to commit 29ac73326aabef2e728122195543c4de335e4283
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for ws3: filename=ws3-0.0.1.post3-py2.py3-none-any.whl size=59108 sha256=d6c9bd82c65ea4e2beecf0094a5eb13c543373a3e19c9c7b9de97d5cd1ce5968
  Stored in directory: /tmp/pip-ephem-wheel-cache-q7wooybs/wheels/58/7f/fd/fffd2465004750dc9a227e6504c396c7076e389e3e2a8e6412
Successfully built ws3
Note: you may need to restart the kernel to use updated packages.


You can test to make sure that your kernel is now picking up the venv-sandboxed version of the package by importing the package and inspecting the value of the package `__path__` attribute.

In [2]:
import ws3
ws3.__path__

['/home/gep/projects/ws3/.venv/lib/python3.12/site-packages/ws3']

You can also use the "%pip" magic command to uninstall packages from your venv. 

For example to uninstall the ws3 package we installed earlier in the "foo" venv, run the following command from inside the kernel. Note the "-y" flag, which skips asking for user confirmation before uninstalling stuff (else the kernel will hang forever waiting for your response, which you cannot provide from inside a Jupyter notebook or console environment).

In [3]:
%pip uninstall -y ws3

Found existing installation: ws3 0.0.1.post3
Uninstalling ws3-0.0.1.post3:
  Successfully uninstalled ws3-0.0.1.post3
Note: you may need to restart the kernel to use updated packages.


Note also that it is possible to install an "editable" version of a Python package inside your venv-sandboxed kernel by cloning a GitHub repo for the Python package you want to use into your project environment and running the "%pip" magic command with the "-e" flag and point to the local directory containing the cloned copy of the package code.

For example, we can clone the `dev` branch of the `ws3` GitHub repo into our project directory like this.

In [4]:
!git clone https://github.com/gparadis/ws3

fatal: destination path 'ws3' already exists and is not an empty directory.


Then we can configure our venv-sandboxed kernel environment to use that local copy of the code when Python imports `ws3` by running the following magic command (assuming this notebook is running the venv kernel).

In [5]:
%pip install -e ./ws3

Obtaining file:///home/gep/projects/ws3/examples/ws3
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for ws3 (pyproject.toml) ... done
  Created wheel for ws3: filename=ws3-0.0.1.post3-py2.py3-none-any.whl size=1867 sha256=1b36d8dae1612fcb9b53974861f8ee1ce1bd3696fc61fe1dcb8c275b10b42178
  Stored in directory: /tmp/pip-ephem-wheel-cache-6d8dj3f8/wheels/92/4f/40/0e1d2f2584d079a190b79c34d238033826099ccae6b5868838
Successfully built ws3
Note: you may need to restart the kernel to use updated packages.


We can check that this is working the way I intended by importing the ws3 package and inspecting its __path__ attribute.

In [6]:
import ws3
ws3.__path__

['/home/gep/projects/ws3/.venv/lib/python3.12/site-packages/ws3']

If you want to get really fancy, you can set up your notebook to autoreload the local package anytime you modify the source code by adding these two lines of code to the top of your notebook.

In [7]:
%load_ext autoreload
%autoreload 2

This is the basic pattern if you are working on a project that uses `ws3` and anticipate needing to tweak the code (at least a little bit) to get it to do _exactly_ what you need it to do for your particular project.